## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ipaddress import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,f1_score,auc,roc_curve,roc_auc_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout, Flatten, SpatialDropout1D, LSTM,Input, Concatenate, Reshape, InputLayer, concatenate
from tensorflow.keras.regularizers import l1, l2, l1_l2
import keras
import re
import tldextract
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='white') 

/home/lab_services_student/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-10-13 11:13:22.045492: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-13 11:13:22.181499: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 11:13:23.283958: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 11:13:24.472412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting

## Data Transforming

In [2]:

data=pd.read_csv('cleaned_data.csv')
df=data[['url','status']]
data=data.drop('url',axis=1)

In [3]:
data.head()

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,has_subdomain,is_HTTPS,depth,top_domain_encoded,domain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,0,1,225,111707
1,0,36,0,6,2,0,0,0,12,0,1,0,1,225,111707
2,0,12,0,5,1,0,0,0,5,5,0,0,1,878,10
3,0,35,0,4,2,0,0,0,12,0,1,0,1,225,111707
4,0,20,0,5,2,0,0,0,8,0,1,0,1,225,291146


In [4]:
y=data['status']
vectorizer = TfidfVectorizer(analyzer='char')
X=vectorizer.fit_transform(df.url)

In [5]:
X.shape

(798463, 266)

In [6]:
X=X.toarray()
print(X.shape)
X=X.reshape(X.shape[0],14,19)

(798463, 266)


In [7]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=54)



In [8]:
X_train.shape

(638770, 14, 19)

## LSTM

In [9]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)
Lstm  = Sequential()
Lstm.add(LSTM(128,input_shape=(14,19),return_sequences=False))
Lstm.add(Dropout(0.2))
Lstm.add(Dense(24,activation='relu'))
Lstm.add(Dense(1,activation ='sigmoid'))
# Lstm.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))

In [10]:
Lstm.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [11]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyL=Lstm.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)


Epoch 1/30
19962/19962 - 187s - 9ms/step - accuracy: 0.8741 - loss: 0.2950 - precision: 0.8574 - recall: 0.9171 - val_accuracy: 0.9039 - val_loss: 0.2363 - val_precision: 0.8829 - val_recall: 0.9455
Epoch 2/30
19962/19962 - 180s - 9ms/step - accuracy: 0.9087 - loss: 0.2251 - precision: 0.8948 - recall: 0.9398 - val_accuracy: 0.9168 - val_loss: 0.2062 - val_precision: 0.9026 - val_recall: 0.9463
Epoch 3/30
19962/19962 - 265s - 13ms/step - accuracy: 0.9179 - loss: 0.2036 - precision: 0.9060 - recall: 0.9445 - val_accuracy: 0.9221 - val_loss: 0.1924 - val_precision: 0.9074 - val_recall: 0.9514
Epoch 4/30
19962/19962 - 321s - 16ms/step - accuracy: 0.9231 - loss: 0.1914 - precision: 0.9123 - recall: 0.9472 - val_accuracy: 0.9259 - val_loss: 0.1830 - val_precision: 0.9190 - val_recall: 0.9445
Epoch 5/30
19962/19962 - 310s - 16ms/step - accuracy: 0.9270 - loss: 0.1825 - precision: 0.9173 - recall: 0.9491 - val_accuracy: 0.9281 - val_loss: 0.1784 - val_precision: 0.9244 - val_recall: 0.9424
Ep

In [12]:
results = Lstm.evaluate(X_test, y_test, verbose=2)
y_pred_prob = Lstm.predict(X_test)  # Get the predicted probabilities
y_pred = (y_pred_prob > 0.5).astype(int)  
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)  
# Assuming you compiled with accuracy, precision, and recall
print(f"Restored Model Test Loss: {results[0]}")
print(f"Restored Model Test Accuracy: {results[1]}")
print(f"Restored Model Test Precision: {results[2]}")
print(f"Restored Model Test Recall: {results[3]}")
print(f"F1 Score: {f1}")
print(f"ROC-AUC Score: {roc_auc}")

4991/4991 - 21s - 4ms/step - accuracy: 0.9323 - loss: 0.1695 - precision: 0.9312 - recall: 0.9429
4991/4991 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step
Restored Model Test Loss: 0.1694868505001068
Restored Model Test Accuracy: 0.9322950839996338
Restored Model Test Precision: 0.9312041401863098
Restored Model Test Recall: 0.9429271221160889
F1 Score: 0.9370289694696503
ROC-AUC Score: 0.9815364521760407
